# Splatoon2のシーン分析

## モチベーション

録画を切り出す際の境界検出ができたらハッピー

In [1]:
import sys
import os
import shutil
import random
import tensorflow as tf
import keras
from keras import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

print(sys.version)
# print(sys.executable)
print(tf.test.is_built_with_cuda())
tf.test.gpu_device_name()

Using TensorFlow backend.


3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
True


'/device:GPU:0'

In [2]:
# 環境変数とか

# 元データ保存先
dataset_base_path = '.\\splat-scene-dataset'
dataset_split_base_path = '.\\dataset'
tensorboard_log_path = '.\\tflog'

# データセットの分離比率
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# 画像設定
input_size = (80, 45)
input_shape = (80, 45, 3)

# データ関係
batch_size = 4
categories_n = 17

dataset_train_path = os.path.join(dataset_split_base_path, 'train')
dataset_val_path   = os.path.join(dataset_split_base_path, 'val')
dataset_test_path  = os.path.join(dataset_split_base_path, 'test')
pathes = [dataset_train_path, dataset_val_path, dataset_test_path]

ratios = [train_ratio, val_ratio, test_ratio]

In [48]:
# Debugするたびに結果が変わるのはひとまず避けたい
random.seed(0)

# learn/validate/testに分離
def split_dataset(src_base_path, dst_base_path, pathes, ratios, debug=False):
    # すでに作成されている場合は一旦削除
    if (os.path.exists(dst_base_path)):
        shutil.rmtree(dst_base_path)
    # ディレクトリ作成
    os.mkdir(dst_base_path)
    for p in pathes:
        os.mkdir(p)
    # ディレクトリ一覧取得
    categories = list(filter(lambda x: 
                             os.path.isdir(os.path.join(src_base_path, x)) 
                             and not(x.startswith('.')),
                     os.listdir(dataset_base_path)))
    categories_n = len(categories) # 返却値にしてあげる
    print(categories)
    # 順番にコピーしてく
    for c in categories:
        files = os.listdir(os.path.join(src_base_path, c))
        files_count = len(files)
        random.shuffle(files)
        
        ratio_sum = sum(ratios)
        take_count = [int(files_count * (r / ratio_sum)) for r in ratios]
        print(c, files_count, take_count)

        for p in pathes:
            dst = os.path.join(p, c)
            if debug:
                print('Mkdir {}'.format(dst))
            os.mkdir(dst)
            
        count = 0
        for t, p in zip(take_count, pathes):
            target_files = files[count:count + t]
            print(c, t, p, len(target_files))
            src = [os.path.join(src_base_path, c, tf) for tf in target_files]
            dst = [os.path.join(p, c, tf) for tf in target_files]
            for s, d in zip(src, dst):
                # print('Copy {} -> {}'.format(s, d))
                shutil.copyfile(s, d)
            count = t
        if debug:
            print('{} copy {} files'.format(c, count))
    return categories_n
        
split_dataset(dataset_base_path, dataset_split_base_path, pathes, ratios)

['battle', 'battle_finish', 'battle_loby', 'battle_matching', 'battle_result', 'battle_rule', 'battle_start', 'loading', 'menu', 'other', 'salmon', 'salmon_lobby', 'salmon_matching', 'salmon_miss', 'salmon_result', 'salmon_start', 'weapon_select']
battle 9558 [5734, 1911, 1911]
Mkdir .\dataset\train\battle
Mkdir .\dataset\val\battle
Mkdir .\dataset\test\battle
battle 5734 .\dataset\train 5734
battle 1911 .\dataset\val 1911
battle 1911 .\dataset\test 1911
battle_finish 215 [129, 43, 43]
Mkdir .\dataset\train\battle_finish
Mkdir .\dataset\val\battle_finish
Mkdir .\dataset\test\battle_finish
battle_finish 129 .\dataset\train 129
battle_finish 43 .\dataset\val 43
battle_finish 43 .\dataset\test 43
battle_loby 280 [168, 56, 56]
Mkdir .\dataset\train\battle_loby
Mkdir .\dataset\val\battle_loby
Mkdir .\dataset\test\battle_loby
battle_loby 168 .\dataset\train 168
battle_loby 56 .\dataset\val 56
battle_loby 56 .\dataset\test 56
battle_matching 1376 [825, 275, 275]
Mkdir .\dataset\train\battle_m

17

In [3]:
# イメージをいい感じに読み込んでもらう
def create_generator(path,
                     target_size,
                     batch_size,
                     class_mode = 'categorical'):
    print(path)
    dg = ImageDataGenerator(rescale=1/255.0)
    gen = dg.flow_from_directory(path, 
                                 target_size=target_size,
                                 batch_size=batch_size,
                                 class_mode=class_mode,
                                 shuffle=True)
    return (dg, gen)

(train_dg, train_gen) = create_generator(dataset_train_path, target_size=input_size, batch_size=batch_size)
(val_dg,   val_gen)   = create_generator(dataset_val_path, target_size=input_size, batch_size=batch_size)
(test_dg,  test_gen)  = create_generator(dataset_test_path, target_size=input_size, batch_size=batch_size)

.\dataset\train
Found 10231 images belonging to 17 classes.
.\dataset\val
Found 3406 images belonging to 17 classes.
.\dataset\test
Found 3406 images belonging to 17 classes.


In [4]:
# VGGの転移学習モデルを作る
from keras.applications.vgg16 import VGG16

def create_vgg16_base_model(categories_n):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    # VGG16自体は学習できないように凍結しておく
    for l in base_model.layers:
        l.trainable = False
    # あとにレイヤを追加
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(categories_n, activation='softmax')(x)
    return Model(inputs= base_model.input, outputs=x)
    
model = create_vgg16_base_model(categories_n)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80, 45, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 80, 45, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 80, 45, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 40, 22, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 40, 22, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 40, 22, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 20, 11, 128)       0         
__________

In [5]:
# モデルのコンパイル
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
# 学習する
tb_cb = TensorBoard(log_dir=tensorboard_log_path)
checkpointer = ModelCheckpoint(filepath='./best.hdf5', verbose=1, save_best_only=True, period=10, save_weights_only=False)

history = model.fit_generator(
    train_gen,
    epochs = 500,
    verbose = 1,
    validation_data=val_gen,
    validation_steps=128,
    callbacks=[tb_cb, checkpointer],
)
model.save('model.h5')
history

Epoch 1/500
2558/2558 [==============================] - 75s 29ms/step - loss: 1.2096 - acc: 0.6430 - val_loss: 0.9955 - val_acc: 0.7173
Epoch 2/500
2558/2558 [==============================] - 62s 24ms/step - loss: 0.8936 - acc: 0.7538 - val_loss: 0.7982 - val_acc: 0.7813
Epoch 3/500
2558/2558 [==============================] - 62s 24ms/step - loss: 0.7489 - acc: 0.7885 - val_loss: 0.6983 - val_acc: 0.8048
Epoch 4/500
2558/2558 [==============================] - 62s 24ms/step - loss: 0.6543 - acc: 0.8116 - val_loss: 0.6118 - val_acc: 0.8332
Epoch 5/500
2558/2558 [==============================] - 61s 24ms/step - loss: 0.5844 - acc: 0.8324 - val_loss: 0.5626 - val_acc: 0.8438
Epoch 6/500
2558/2558 [==============================] - 60s 23ms/step - loss: 0.5325 - acc: 0.8466 - val_loss: 0.5107 - val_acc: 0.8555
Epoch 7/500
2558/2558 [==============================] - 60s 23ms/step - loss: 0.4932 - acc: 0.8601 - val_loss: 0.4728 - val_acc: 0.8705
Epoch 8/500
2558/2558 [==================

In [7]:
# テストデータで性能確認
result = model.evaluate_generator(
    test_gen,
    verbose=1
)
result

852/852 [==============================] - 16s 19ms/step


[0.016905705805647167, 0.9991192014092778]